In [10]:
%pip install pyautogen


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from typing_extensions import Annotated

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST.json")
llm_config = {"config_list": config_list, "stream": True}

In [12]:
task = """KB국민은행 주가 추이에 대해 에 대해 간결하지만 매력적인 블로그 게시물을 작성해줘"""

In [13]:
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    귀하는 통찰력 있고 매력적인 기사로 유명한 전문 작가입니다.
    복잡한 개념을 설득력 있는 이야기로 전환합니다.
    사용자의 피드백을 바탕으로 콘텐츠의 품질을 개선해야 합니다.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config={"config_list": config_list},
    system_message="""
    귀하는 기준에 대한 철저함과 헌신으로 유명한 비평가입니다.
    귀하의 임무는 콘텐츠에 유해한 요소나 규정 위반 사항이 있는지 면밀히 검토하여 다음과 같은 사항을 확인하는 것입니다.
    모든 자료가 필수 가이드라인에 부합하는지 확인합니다.
    코드의 경우
    """,
)

In [14]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"다음 글에 대해 반성하고 비평을 제공합니다. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


user_proxy.register_nested_chats(
    [{"recipient": critic, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,  # condition=my_condition,
)

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

User (to Writer):

KB국민은행 주가 추이에 대해 에 대해 간결하지만 매력적인 블로그 게시물을 작성해줘

--------------------------------------------------------------------------------
Writer (to User):

**한눈에 보는 KB국민은행 주가 추이: 경제의 바로미터**

주식시장은 경제의 기상도라 불립니다. 그중에서도 은행주는 금융시장의 건강 상태를 보여주는 중요한 지표 역할을 합니다. 오늘은 KB국민은행의 주가 추이를 통해 한국 경제의 단면을 들여다봅니다.

### 2021년: 회복의 시간
2021년 초, 팬데믹의 충격에서 벗어나기 시작한 금융 시장은 서서히 회복세를 보였습니다. KB국민은행의 주가도 예외는 아니었죠. 백신 보급과 함께 경제 재개가 가시화되면서 금융주가 자신감을 얻었습니다. 이 시기 KB국민은행의 주가는 안정적인 상승 곡선을 그리며 투자자에게 희망을 안겨주었습니다.

### 2022년: 불확실성 속의 견고함
2022년, 글로벌 경제는 예상치 못한 변수에 노출되었습니다. 우크라이나 전쟁과 공급망 위기가 대표적이었죠. 그러나 KB국민은행은 이러한 불확실성 속에서도 비교적 견고한 모습을 보였습니다. 안정적인 재무구조와 선진화된 리스크 관리 덕분입니다. 주가는 큰 폭의 변동 없이 안정적으로 유지되었고, 이는 기관 투자가들과 개인 투자가 양측에게 긍정적인 신호로 작용했습니다.

### 2023년: 도전과 기회
2023년 현재, KB국민은행은 새로운 도전에 직면해 있습니다. 금리 인상과 인플레이션 압박으로 인해 금융업계에 변화의 바람이 불고 있습니다. 하지만 KB국민은행은 디지털 혁신과 글로벌 시장 확대를 통해 지속가능한 성장의 기회를 모색하고 있습니다. 주가는 이러한 변화에 빠르게 반응하며 다시 한 번 상승세를 기록하고 있습니다.

### 결론: KB국민은행의 강점과 전망
KB국민은행의 주가 추이는 한국 경제의 바로미터로 작용하며, 금융시장의 전반적인 건강 상태를